# Meituan AppSpider
- Author: Yiran Jing

Build Meituan AppSpider to collect customer comments of 'Macaroni of Zhang grandpa' chain restaurants

Scraping content:
1. Store location（店铺地址）
2. Score of store （店铺评分）
3. Consumption per person （人均消费）
4. Business Hours （营业时间）
5. Customer comments (网友点评)
6. Recommended dishes (推荐菜)


Reference: 

1. https://github.com/hahaha108/meituanAppSpider
2. https://blog.csdn.net/slphahaha/article/details/81145404


In [1]:
import requests
import json
from lxml import etree
import pandas as pd
import re
import math
import urllib
import urllib.request as urllib2
import xlsxwriter
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
import csv

### ShenZhen Area
- 目前美团有两个店铺，但是暂时都无评论

### Shanghai Area

- 爬取上海地区张爷爷空心面的美团信息
- 上海地区有五个连锁店铺,但目前只有两个店铺有用户评价，所以暂时只抓了两个店铺

In [2]:
from spider_develop_shanghai import MeituanSpider_shanghai

# save_mode ：txt存储为txt文件，csv存储为csv文件，无输入默认为csv
spider_shanghai = MeituanSpider_shanghai()
taglist_1, commentslist_1 = spider_shanghai.run(1000) # max read 10 pages, 10 comments in each page

Spider is ready for shanghai area


### Nanjing Area
- 目前美团有两个店铺
- 只有一个店铺有一则评论, 暂不抓取

### TaiYuan Area
- 目前美团有一家店铺：（茂业中心店）

In [3]:
from spider_develop_taiyuan import MeituanSpider_taiyuan

# save_mode ：txt存储为txt文件，csv存储为csv文件，无输入默认为csv
spider_taiyuan = MeituanSpider_taiyuan()
taglist_2, commentslist_2 = spider_taiyuan.run(1000)

Spider is ready for Taiyuan area


### TaiCang area
- 目前美团有一家店铺：

In [4]:

from spider_develop_taicang import MeituanSpider_taicang

# save_mode ：txt存储为txt文件，csv存储为csv文件，无输入默认为csv
spider_taicang = MeituanSpider_taicang()
taglist_3, commentslist_3 = spider_taicang.run(1000)

Spider is ready for Taicang area


## Save result to csv

In [5]:
taglist = taglist_1 + taglist_2 + taglist_3
commentslist = commentslist_1 + commentslist_2 + commentslist_3

In [15]:
def Write_to_csv(fieldKey, file_name,datalist):
    with open(file_name, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames= fieldKey)
        writer.writeheader()
        for data in datalist:
            writer.writerow(data)

In [16]:
## Write user comments detail to csv
fieldkey = ['店名',"用户名", '平均消费','星级','菜单','评价时间','用餐结束时间','评价']
file_name = '../../../data/meituanZhangYeyeInfos/Zhangyeye_Comments.csv'
Write_to_csv(fieldkey, file_name, commentslist)

In [12]:
## Write tag retails to csv file
fieldKey = ['标签','累计数量']
file_name = "../../../data/meituanZhangYeyeInfos/Zhangyeye_Tags.csv"
Write_to_csv(fieldKey, file_name, taglist)

In [15]:
df  = pd.read_csv("../../../data/meituanZhangYeyeInfos/Zhangyeye_Tags.csv")
## merge counts if they have same tags 
aggregation_functions = {'标签': 'first','累计数量': 'sum'}
df_new = df.groupby(df['标签']).aggregate(aggregation_functions)
df_new.to_csv('../../../data/meituanZhangYeyeInfos/Zhangyeye_Tags.csv',index=False,encoding='utf_8_sig') # utf_8_sig 为了避免中文乱码